In [ ]:
correct = ["գիրք", "տետր", "պատուհան", "Կարինե", "կարտոֆիլ", "գրիչ", "անհավանական", "քննություն", "արհեստական", "բնական", "համակարգիչ", "մատիտ", "սեղան", "գրատախտակ", "ականջակալ", "քաղաք", "հանրապետություն", "թուղթ", "գրադարան", "վարձակալություն", "գրագետ"]
correct = ["գիրտ", "տետհ", "պատուեան", "ուարինե", "կարտոֆիլ", "գրիչ", "անհավանական", "քննություն", "արհեստական", "բնական", "համակարգիչ", "մատիտ", "սեղան", "գրատախտակ", "ականջակալ", "քաղաք", "հանրապետություն", "թուղթ", "գրադարան", "վարձակալություն", "գրագետ"]


In [115]:
checker = SpellChecker()

In [116]:
len(checker.WORDS)

32198

In [117]:
checker.correction("receit")


        Edit of
            content = receit
            edit = receipt
            type = insert
            edit word probaility = 1.1653078877898144e-05
            keyboard_distance = 2
            levenshtein distance = 1
            previous edit = 
                None
        

In [118]:
candidates = list(checker.candidates("receit")) # receipt
candidates.sort(key = lambda x: checker.P(x), reverse = True)
candidates.sort(key = lambda x: x.keyboard_distance)
candidates.sort(key = lambda x: x.edit_type.value)
candidates

[
         Edit of
             content = receit
             edit = receipt
             type = insert
             edit word probaility = 1.1653078877898144e-05
             keyboard_distance = 2
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = receit
             edit = deceit
             type = replace
             edit word probaility = 3.585562731660967e-06
             keyboard_distance = 2
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = receit
             edit = recent
             type = replace
             edit word probaility = 4.750870619450781e-05
             keyboard_distance = 4
             levenshtein distance = 1
             previous edit = 
                 None
         ]

In [108]:
candidates = list(checker.candidates("receit")) # receipt
[(edit.probability(), edit.edit) for edit in candidates]

[(0.0, 'deceit'), (0.0, 'recent'), (0.0, 'receipt')]

In [109]:
checker.P(Edit("recent"))

TypeError: __init__() missing 4 required positional arguments: 'edit', 'edit_type', 'edit_word_probability', and 'keyboard_distance'

In [110]:
checker.correction("mre")


        Edit of
            content = mre
            edit = more
            type = insert
            edit word probaility = 0.001790092193781738
            keyboard_distance = 4
            levenshtein distance = 1
            previous edit = 
                None
        

In [111]:
candidates = checker.candidates("mre") # more
candidates

{
         Edit of
             content = mre
             edit = are
             type = replace
             edit word probaility = 0.0032538981789823275
             keyboard_distance = 7
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = ere
             type = replace
             edit word probaility = 8.963906829152417e-07
             keyboard_distance = 6
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = ire
             type = replace
             edit word probaility = 8.963906829152417e-07
             keyboard_distance = 3
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = mare
             type = insert
             edit word pro

In [87]:
checker.correction("evar")


        Edit
            content = ever
            type = replace
            previous edit = 
                None
            keyboard_distance = 3
        

In [35]:
candidates = checker.candidates("evar") # ever
candidates

{
         Edit
             content = ear
             type = delete
             previous edit = 
                 None
             keyboard_distance = 2
         ,
 
         Edit
             content = eva
             type = delete
             previous edit = 
                 None
             keyboard_distance = 0
         ,
 
         Edit
             content = ever
             type = replace
             previous edit = 
                 None
             keyboard_distance = 3
         }